In [1]:
#imports
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time
import requests
from bs4 import BeautifulSoup
import re
import config
import json
import os
from urllib.parse import urlparse
import csv

In [2]:
# setup chromedriver
driver = webdriver.Chrome()

# open the webpage
driver.get("https://www.instagram.com/")

In [3]:
# target username and password
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))

# enter username and password
username.clear()
username.send_keys(config.username)
password.clear()
password.send_keys(config.password)

# target and click login button
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

In [4]:
# wait 10 sec for search button
search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'svg[aria-label="Pesquisa"]')))

# click search button
search_button.click()

In [5]:
# target searchbox
searchbox = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[aria-label="Entrada da pesquisa"]')))
searchbox.clear()

# search for @target
keyword = config.target
searchbox.send_keys(keyword)

In [6]:
# click target
time.sleep(5)
result = driver.find_element(By.XPATH, f'//span[text()="{keyword}"]')
result.click()

In [7]:
# get the initial page height
initial_height = driver.execute_script("return document.body.scrollHeight")

# create a list to store htmls
soups = []

# scroll loop
while True:

    # scroll down to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # wait for a moment to allow new content to load (adjust as needed)
    time.sleep(5)

    # parse the html
    html = driver.page_source

    # create a beautifulsoup object from the scraped html
    soups.append(BeautifulSoup(html, 'html.parser'))

    # get the current page height
    current_height = driver.execute_script("return document.body.scrollHeight")

    if current_height == initial_height:
        break # exit the loop when you cant scroll further

    # update the initial heightfor the next iteration
    initial_height = current_height
    
    

In [8]:
# list to store the post image urls
post_urls = []

# loop through soup elements
for soup in soups:

    #find all image elements that match the specific class in the current soup
    elements = soup.find_all('a', class_='x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz _a6hd')

    # extract the href attributes and filterurls that start with "/p/" or "/reel/"
    post_urls.extend([element['href'] for element in elements if element['href'].startswith(("/p/", "/reel/"))])

# convert the list to a set to remove duplicates
unique_post_urls = list(set(post_urls))

In [9]:
# create a list to store the json for each post
json_list = []

# define the query parameters to add
query_parameters = "?__a=1&__d=dis"

# go through all urls
for url in unique_post_urls:

    # error handling
    try:

        # get the current url of the page
        current_url = driver.current_url

        # append the query parameters to the current url
        modified_url = "https://instagram.com/" + url + query_parameters

        # get url
        driver.get(modified_url)

        # wait for a moment to allow new content to load
        time.sleep(1)

        # find the <pre> tag containing the json data
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//pre'))
        )
        pre_tag = driver.find_element(By.XPATH, '//pre')

        # extract the json data from the <pre> tag
        json_script = pre_tag.text

        # parse the json data
        json_parsed = json.loads(json_script)

        # add json to the list
        json_list.append(json_parsed)

    # error handling
    except (NoSuchElementException, TimeoutException, json.JSONDecodeError) as e:
        print(f"Error processing URL {url}: {e}")
        

In [10]:
# lists to store urls and corresponding dates
all_urls = []
all_dates = []

# iterate through each json data in the list
for json_data in json_list:

    # extract the list from the 'items' key
    item_list = json_data.get('items', [])

    # iterate through each item in the 'items' list
    for item in item_list:

        # extract the date the item was taken
        date_taken = json_data.get('taken_at')

        # check if 'carousel_media' is present
        carousel_media = item.get('carousel_media', [])

        # iterate through each media in the 'carousel_media' list
        for media in carousel_media:

            # extract the image URL from the media
            image_url = media.get('image_versions2', {}).get('candidates', [{}])[0].get('url')

            # check if the image_url field id found inside the 'carousel_media' list
            if image_url:

                # add the image url and corresponding date to the lists
                all_urls.append(image_url)
                all_dates.append(date_taken)
                print("carousel image added")

            # extract the video url from the media
            video_versions = media.get('video_versions', [])
            if video_versions:
                video_url = video_versions[0].get('url')
                if video_url:

                    # add the video url and corresponding date to the list
                    all_urls.append(video_url)
                    all_dates.append(date_taken)
                    print("carousel video added")

        # handle cases of unique image, instead of carousel
        image_url = item.get('image_versions2', {}).get('candidates', [{}])[0].get('url')
        if image_url:

            # add the image url and corresponding date to the lists
            all_urls.append(image_url)
            all_dates.append(date_taken)
            print(f"single image added")

        # check if 'video_versions' key exists
        video_versions = item.get('video_versions', [])
        if video_versions:
            video_url = video_versions[0].get('url')
            if video_url:

                # add the videourl and correspondig date to the lists
                all_urls.append(video_url)
                all_dates.append(date_taken)
                print(f"single video added")

print(len(all_urls))

single image added
single video added
single image added
single video added
single image added
single video added
single image added
single video added
carousel image added
carousel image added
single image added
single image added
single video added
carousel image added
carousel image added
carousel image added
carousel image added
carousel image added
carousel image added
carousel image added
carousel image added
carousel image added
carousel image added
single image added
carousel image added
carousel image added
carousel image added
single image added
single image added
carousel image added
carousel image added
single image added
carousel image added
carousel image added
carousel image added
single image added
36


In [12]:
all_dates

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [11]:
# create a directory to store downloaded files
download_dir = keyword
os.makedirs(download_dir, exist_ok=True)

# create subfolders for images and videos
image_dir = os.path.join(download_dir, "images")
video_dir = os.path.join(download_dir, "videos")
os.makedirs(image_dir, exist_ok=True)
os.makedirs(video_dir, exist_ok=True)

# initialize counters for images and videos
image_counter = 1
video_counter = 1

# iterate through urls in the all_urls list and download_media
for index, url in enumerate(all_urls, 0):
    response = requests.get(url, stream=True)

    # extract file extension from the url
    url_path = urlparse(url).path
    file_extension = os.path.splitext(url_path)[1]

    # determine the file name based on the url
    if file_extension.lower() in {'.jpeg', '.jpg', '.png', '.gif'}:
        file_name = f"{all_dates[index]}-img-{image_counter}.png"
        destination_folder = image_dir
        image_counter += 1
    elif file_extension.lower() in {'.mp4', '.avi', '.mkv', '.mov'}:
        file_name = f"{all_dates[index]}-vid-{video_counter}.mp4"
        destination_folder = video_dir
        video_counter += 1
    else:
        # default to the main donwload directory for other file types
        file_name = f"{all_dates[index]}{file_extesion}"
        destination_folder = download_dir

    # save the file to the appropriate folder
    file_path = os.path.join(destination_folder, file_name)

    # write the content of the response to the file
    with open(file_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                file.write(chunk)

    print(f"Downloaded: {file_path}")

# print a message indicating the number of downloaded files and the download directory
print(f'Downloaded {len(all_urls)} files to {download_dir}')
    

Downloaded: pharrell\images\None-img-1.png
Downloaded: pharrell\videos\None-vid-1.mp4
Downloaded: pharrell\images\None-img-2.png
Downloaded: pharrell\videos\None-vid-2.mp4
Downloaded: pharrell\images\None-img-3.png
Downloaded: pharrell\videos\None-vid-3.mp4
Downloaded: pharrell\images\None-img-4.png
Downloaded: pharrell\videos\None-vid-4.mp4
Downloaded: pharrell\images\None-img-5.png
Downloaded: pharrell\images\None-img-6.png
Downloaded: pharrell\images\None-img-7.png
Downloaded: pharrell\images\None-img-8.png
Downloaded: pharrell\videos\None-vid-5.mp4
Downloaded: pharrell\images\None-img-9.png
Downloaded: pharrell\images\None-img-10.png
Downloaded: pharrell\images\None-img-11.png
Downloaded: pharrell\images\None-img-12.png
Downloaded: pharrell\images\None-img-13.png
Downloaded: pharrell\images\None-img-14.png
Downloaded: pharrell\images\None-img-15.png
Downloaded: pharrell\images\None-img-16.png
Downloaded: pharrell\images\None-img-17.png
Downloaded: pharrell\images\None-img-18.png
Do